In [1]:
import pyabf
import numpy as np
from scipy import signal
import csv

# Define the project path and a list of 5 ABF file names
project_path = 'C:\\Users\\ikmor\\OneDrive\\Documents\\SNR AND OBR pilot study\\'
abf_files = [
    f'{project_path}PTX10 CNQX A3.abf',
    f'{project_path}PTX8R3 ACCIN.abf',
    f'{project_path}PTX7L2 ACCOM.abf',
    f'{project_path}PTX09R5 ACCOM3.abf',
    f'{project_path}PTX6L2 ACCOM.abf',
    f'{project_path}CNQX1L3 A3.abf'
]

# Initialize a 2D list to store average time between peaks for each sweep across all files
sweep_peak_intervals = [[] for _ in range(10)]  # 10 sweeps

LJP = 13  # Liquid Junction Potential in mV

# Loop through each ABF file
for abf_file in abf_files:
    # Load the ABF file
    abf = pyabf.ABF(abf_file)
    
    # Define sweep numbers
    sweep_numbers = range(10)  # 10 sweeps

    # Loop through each sweep
    for sweep in sweep_numbers:
        abf.setSweep(sweep, channel=0)
        
        # Define the time segment for analysis (in seconds)
        start_time = 2.5  # Start time in seconds
        end_time = 7.5    # End time in seconds

        # Extract voltage (sweepY) and time (sweepX)
        voltage = abf.sweepY - LJP  # Voltage in mV
        times = abf.sweepX    # Time in seconds

        # Find indices corresponding to the desired time segment
        start_index = np.argmin(np.abs(times - start_time))
        end_index = np.argmin(np.abs(times - end_time))

        # Extract the specific portion of the data for analysis
        segment_voltage = voltage[start_index:end_index]
        segment_times = times[start_index:end_index]

        # Find local peaks above 0mV
        peaks, _ = signal.find_peaks(segment_voltage, height=0)

        # Calculate time intervals between consecutive peaks
        if len(peaks) > 1:
            peak_intervals = np.diff(segment_times[peaks])
            average_interval = np.mean(peak_intervals)
        else:
            average_interval = np.nan  # Not enough peaks to calculate intervals

        # Store the average interval for this sweep
        sweep_peak_intervals[sweep].append(average_interval)

# Calculate average time intervals for each sweep
average_peak_intervals = [np.nanmean(intervals) for intervals in sweep_peak_intervals]

# Calculate standard error for each sweep
standard_errors = [
    np.nanstd(intervals, ddof=1) / np.sqrt(np.sum(~np.isnan(intervals))) if np.sum(~np.isnan(intervals)) > 1 else np.nan 
    for intervals in sweep_peak_intervals
]

# Output results to CSV
output_file = f'{project_path}average_peak_intervals_with_errors.csv'
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Sweep', 'Average Time Between Peaks (s)', 'Standard Error'])
    for sweep, (avg_interval, std_err) in enumerate(zip(average_peak_intervals, standard_errors)):
        writer.writerow([sweep + 1, avg_interval, std_err])

print(f"Results with standard errors have been saved to {output_file}")

Results with standard errors have been saved to C:\Users\ikmor\OneDrive\Documents\SNR AND OBR pilot study\average_peak_intervals_with_errors.csv


C:\Users\ikmor\AppData\Local\Temp\ipykernel_14476\2315623866.py:64: RuntimeWarning: Mean of empty slice
  average_peak_intervals = [np.nanmean(intervals) for intervals in sweep_peak_intervals]
